In [179]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import LogisticRegression, SGDClassifier
import math

# Feature engineering

In [180]:
raw_dataset = pd.read_csv('https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv', index_col=False )

raw_dataset['Fam_size'] = raw_dataset['Siblings/Spouses Aboard'] + raw_dataset['Parents/Children Aboard']

raw_dataset['Name_Len'] = [ len(i) for i in raw_dataset["Name"] ]

raw_dataset['Is_alone'] = [ i == 0 for i in raw_dataset["Fam_size"] ]

raw_dataset["Sex"] = [ i == "Male" for i in raw_dataset["Sex"] ]

raw_dataset['Title'] = [ i.split()[0] for i in raw_dataset["Name"]]
raw_dataset = pd.concat([raw_dataset, pd.get_dummies(raw_dataset['Title'])], axis=1)

raw_dataset = pd.concat([raw_dataset, pd.get_dummies(raw_dataset['Pclass'])], axis=1)

# Dropping the useless features
raw_dataset.drop('Name', axis=1, inplace=True)
raw_dataset.drop('Pclass', axis=1, inplace=True)
raw_dataset.drop('Siblings/Spouses Aboard', axis=1, inplace=True)
raw_dataset.drop('Parents/Children Aboard', axis=1, inplace=True)
raw_dataset.drop('Title', axis=1, inplace=True)

x = raw_dataset.drop('Survived', axis=1).to_numpy()
y = raw_dataset['Survived'].to_numpy()


#prepare dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
print(len(x_train))

594


# Debug : Logistic Regression 

In [208]:
clf = LogisticRegression(max_iter=10000, warm_start=True)

# Approche naive :
On incrémente au fil du jeu de donnée pour voir si on obtient un résultat similaire à si on avait tout fait d'un coup

In [209]:
batch_size = 20
iter = math.floor(len(x_train)/batch_size)

print('itérrations :', iter)

for i in range(iter):
    x = x_train[batch_size*i : batch_size*(i+1)]
    y = y_train[batch_size*i : batch_size*(i+1)]
    clf.fit(x, y)
    print("itération", i, ":", clf.score(x_test, y_test))

print("-------------")

x = x_train[0 : iter*batch_size]
y = y_train[0 : iter*batch_size]
clf.fit(x, y)
print("entrainement sur tout le jeu de donnée d'un coup :",clf.score(x_test, y_test))

itérrations : 29
itération 0 : 0.7064846416382252
itération 1 : 0.7030716723549488
itération 2 : 0.7201365187713311
itération 3 : 0.6723549488054608
itération 4 : 0.6313993174061433
itération 5 : 0.7508532423208191
itération 6 : 0.7098976109215017
itération 7 : 0.658703071672355
itération 8 : 0.726962457337884
itération 9 : 0.7337883959044369
itération 10 : 0.7508532423208191
itération 11 : 0.5665529010238908
itération 12 : 0.757679180887372
itération 13 : 0.6484641638225256
itération 14 : 0.689419795221843
itération 15 : 0.7610921501706485
itération 16 : 0.6860068259385665
itération 17 : 0.7713310580204779
itération 18 : 0.7167235494880546
itération 19 : 0.6928327645051194
itération 20 : 0.6655290102389079
itération 21 : 0.7440273037542662
itération 22 : 0.7133105802047781
itération 23 : 0.6313993174061433
itération 24 : 0.6552901023890785
itération 25 : 0.7337883959044369
itération 26 : 0.78839590443686
itération 27 : 0.6621160409556314
itération 28 : 0.6791808873720137
-------------

# Résultats :

ca n'a pas l'air de fonctionner, on a l'impression que le classifier est réinitialisé et réentrainé à chaque mini batch

notamment sur ces 3 lignes :
```
itération 10 : 0.7508532423208191
itération 11 : 0.5665529010238908
itération 12 : 0.757679180887372
```
De plus, sur la dernière ittération, les performances semblent loin d'être optimales comparées à l'apprentissage sur l'intégralité du jeu de donnée :
```
itération 28 : 0.6791808873720137
```

```
entrainement sur tout le jeu de donnée d'un coup : 0.7849829351535836
```

Explication possible : Peut-être que sur l'ittération 11 on a affaire a un sous-échantillon qui dévie fortement du jeu de test ?


# Investigation :

Essayons de vérifier les poids entre deux "epochs"

In [252]:
clf = LogisticRegression(max_iter=10000, warm_start=1, random_state = 0)

batch_size = 20

iter = math.floor(len(x_train)/batch_size)

fewer_iter = 5

print('itérrations :', iter)

for i in range(fewer_iter):
    x = x_train[batch_size*i : batch_size*(i+1)]
    y = y_train[batch_size*i : batch_size*(i+1)]
    clf.fit(x, y)
    print("nombre d'itérations avant de converger :",clf.n_iter_)
    print("minibatch", i, ":", clf.score(x_test, y_test))

print("-------------")

x = x_train[0 : fewer_iter*batch_size]
y = y_train[0 : fewer_iter*batch_size]
clf.fit(x, y)
print("nombre d'itérations avant de converger :",clf.n_iter_)
print("entrainement sur tout le jeu de donnée d'un coup :", clf.score(x_test, y_test))

itérrations : 29
nombre d'itérations avant de converger : [108]
minibatch 0 : 0.7064846416382252
nombre d'itérations avant de converger : [154]
minibatch 1 : 0.7030716723549488
nombre d'itérations avant de converger : [146]
minibatch 2 : 0.7201365187713311
nombre d'itérations avant de converger : [114]
minibatch 3 : 0.6723549488054608
nombre d'itérations avant de converger : [167]
minibatch 4 : 0.6313993174061433
-------------
nombre d'itérations avant de converger : [284]
entrainement sur tout le jeu de donnée d'un coup : 0.7406143344709898


# Ci dessus, une cellule ayant un comportement qui, je pensais, devait être similaire

In [260]:
clf = LogisticRegression(max_iter=10000, warm_start=1, random_state=0)

batch_size = 20

iter = math.floor(len(x_train)/batch_size)
fewer_iter = 5

print('itérrations :', iter)

for i in range(fewer_iter):
    x = x_train[0 : batch_size*(i+1)]
    y = y_train[0 : batch_size*(i+1)]
    clf.fit(x, y)
    print("nombre d'itérations avant de converger :",clf.n_iter_)
    print("minibatch", i, ":", clf.score(x_test, y_test))

print("-------------")

x = x_train[0 : fewer_iter*batch_size]
y = y_train[0 : fewer_iter*batch_size]
clf.fit(x, y)
print("nombre d'itérations avant de converger :",clf.n_iter_)
print("entrainement sur tout le jeu de donnée d'un coup :", clf.score(x_test, y_test))

itérrations : 29
nombre d'itérations avant de converger : [108]
minibatch 0 : 0.7064846416382252
nombre d'itérations avant de converger : [147]
minibatch 1 : 0.7098976109215017
nombre d'itérations avant de converger : [194]
minibatch 2 : 0.7201365187713311
nombre d'itérations avant de converger : [193]
minibatch 3 : 0.7201365187713311
nombre d'itérations avant de converger : [150]
minibatch 4 : 0.7440273037542662
-------------
nombre d'itérations avant de converger : [1]
entrainement sur tout le jeu de donnée d'un coup : 0.7440273037542662


# Résultats

Quand on regarde le nombre d'itérations avant de converger dans les deux experiences on remarque plusieurs choses.

A chaque minibatch, l'algoritme met un certain temps avant de converger. On en conclut donc que d'un minibatch à l'autre, il y a bien un nouveau minimum a chercher.

Dans le cas des entraitement sur le dataset complet, on a deux cas différents :

Dans le premiers cas, on se situait déjà sur le minimum du dernier minibatch, il y a donc ne nombreuses étapes avant d'arriver sur le minimum global.

Dans le deuxieme cas, on se situe déjà sur le minimum global, il n'y a donc qu'un pas. Cela confirme le fait que rappeller la methode fit() ne réinitialise pas le modele entre deux époch.

# Essayons de réinitialiser le model pour voir l'impact sur le nombre d'itération entre chaque minibatch

In [261]:
batch_size = 20

iter = math.floor(len(x_train)/batch_size)
fewer_iter = 5

print('itérrations :', iter)

for i in range(fewer_iter):
    clf = LogisticRegression(max_iter=10000, warm_start=True, random_state = 0)
    x = x_train[0 : batch_size*(i+1)]
    y = y_train[0 : batch_size*(i+1)]
    clf.fit(x, y)
    print("nombre d'itérations avant de converger :",clf.n_iter_)
    print("minibatch", i, ":", clf.score(x_test, y_test))
    
print("-------------")
x = x_train[0 : fewer_iter*batch_size]
y = y_train[0 : fewer_iter*batch_size]
clf.fit(x, y)
print("nombre d'itérations avant de converger :",clf.n_iter_)
print("entrainement sur tout le jeu de donnée d'un coup :", clf.score(x_test, y_test))
print("-------------")

itérrations : 29
nombre d'itérations avant de converger : [108]
minibatch 0 : 0.7064846416382252
nombre d'itérations avant de converger : [171]
minibatch 1 : 0.7098976109215017
nombre d'itérations avant de converger : [203]
minibatch 2 : 0.7201365187713311
nombre d'itérations avant de converger : [225]
minibatch 3 : 0.7201365187713311
nombre d'itérations avant de converger : [236]
minibatch 4 : 0.7440273037542662
-------------
nombre d'itérations avant de converger : [1]
entrainement sur tout le jeu de donnée d'un coup : 0.7440273037542662
-------------


On constate une légère dégradation. 

Ce qui semble confirmer que les poids n'étaient pas réinitialiser à chaque fit() successif.

# Inutile de faire du séquentiel pure ?

Il semblerait que cette approche n'ai pas d'interet. 
Ou au moins qu'elle n'ai pas d'interet sur ce jeu de données.

A chaque minibatch, l'algoritme converge vers une solution "optimale" (pour ce sub-dataset précis). 

En partant de cette nouvelle solution, le model va chercher à converger vers un nouveau minimum qui est celui du mimibatch actuel.

En séquentiel, les minimum correspondent a des jeux de données plus restreints qui ne peuvent être que moins optimaux que le minimum atteint par le dateset complet pour un probleme donné.

Cette approche n'a donc aucun interet puisqu'elle converge vers le minimum de chaque minibatch, et que le minimum global ne peut pas être atteint.

## Perspectives à explorer :

On pourrait artificiellement limiter max_iter pour empecher le modele de converger à chaque fois.

Ainsi, on pourrait voir si cette approche peut présenter un interet dans le cas ou l'algoritme n'arrive pas à converger.

Cela donnerais des indice sur l'éventuelle utilité d'une approche séquentielle sur des problemes ou des modeles plus compliqués.

# Approche en federated averaging

Chaque partenaire entraine son modele sur un minibatch différent en paralele, on moyene les poids, on test.


In [302]:
clf = LogisticRegression(max_iter=10000, random_state = 0)


n_partner = 4
batch_size = math.floor(len(x_train)/n_partner)

print('nombre de partenairs :', n_partner)

#initialisation de la matrice de poids
coefs = [None]* n_partner
intercepts = [None]* n_partner

for i in range(n_partner):
    x = x_train[batch_size*i : batch_size*(i+1)]
    y = y_train[batch_size*i : batch_size*(i+1)]
    clf.fit(x, y)
    (coefs[i], intercepts[i]) = (clf.coef_,clf.intercept_)
    print("itération", i, ":", clf.score(x_test, y_test))


sum_coefs = coefs[0]*0
sum_intercepts = intercepts[0]*0

for i in range(n_partner):
    sum_coefs = sum_coefs + coefs[i]
    sum_intercepts = intercepts + intercepts[i]
        
    
#averaging
avg_coef = sum_coefs/n_partner
avg_intercepts = sum_intercepts/n_partner

# assemblage du model moyenné :
clf = LogisticRegression()


clf.coef_ = avg_coef
clf.intercept_ = avg_intercepts



nombre de partenairs : 4
itération 0 : 0.7781569965870307
itération 1 : 0.764505119453925
itération 2 : 0.7815699658703071
itération 3 : 0.7952218430034129


# test du modele moyenné et comparaison

In [303]:
print(clf.coef_ == avg_coef, clf.intercept_ == avg_intercepts)


print("test sur modele moyenné :", clf.score(x_test, y_test))

print("-------------")
x = x_train[0 : batch_size*n_partner]
y = y_train[0 : batch_size*n_partner]
clf = LogisticRegression(max_iter=10000, random_state = 0)

clf.fit(x, y)
print("test sur tout le jeu de donnée d'un coup :", clf.score(x_test, y_test))

[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True]] [[ True]
 [ True]
 [ True]
 [ True]]


ValueError: operands could not be broadcast together with shapes (293,1) (4,1) 

In [ ]:
clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=0)

In [191]:
iter = math.floor(len(x_train)/10)

print(iter)

for i in range(iter):
    print(i)
    x = x_train[10*i:10*(i+1)]
    y = y_train[10*i:10*(i+1)]
    clf.partial_fit(x, y, classes=[0,1])
    print(clf.score(x_test, y_test))

print("-------------")

x = x_train[0:iter*10]
y = y_train[0:iter*10]
clf.fit(x, y)
print(clf.score(x_test, y_test))

59
0
0.4641638225255973
1
0.4334470989761092
2
0.6757679180887372
3
0.6109215017064846
4
0.6825938566552902
5
0.6825938566552902
6
0.6040955631399317
7
0.3890784982935154
8
0.45051194539249145
9
0.6996587030716723
10
0.3993174061433447
11
0.5085324232081911
12
0.6143344709897611
13
0.7030716723549488
14
0.6143344709897611
15
0.6109215017064846
16
0.5767918088737202
17
0.6518771331058021
18
0.6109215017064846
19
0.6109215017064846
20
0.7030716723549488
21
0.6791808873720137
22
0.44368600682593856
23
0.6109215017064846
24
0.3890784982935154
25
0.6518771331058021
26
0.6143344709897611
27
0.45051194539249145
28
0.41638225255972694
29
0.6450511945392492
30
0.6450511945392492
31
0.6279863481228669
32
0.6928327645051194
33
0.6860068259385665
34
0.6006825938566553
35
0.689419795221843
36
0.6450511945392492
37
0.6484641638225256
38
0.6416382252559727
39
0.6689419795221843
40
0.6313993174061433
41
0.6825938566552902
42
0.3583617747440273
43
0.658703071672355
44
0.3924914675767918
45
0.6143344709

0.7849829351535836
